# Orderbook 기초 - Intraday Trading 입문

이 노트북에서는 Binance의 실시간 Orderbook 데이터를 수신하고 분석하는 방법을 배웁니다.

## 학습 목표

1. **Orderbook (호가창)** 이해하기
2. **Bid-Ask Spread** 계산하고 해석하기
3. **Mid-price vs Micro-price** 차이 이해하기
4. **Orderbook 히트맵** 시각화하기

---

## 1. 환경 설정

In [2]:
import asyncio
import sys
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



# 프로젝트 모듈 import
sys.path.insert(0, '../src')
from intraday.client import BinanceWebSocketClient, OrderbookSnapshot
from intraday.orderbook import OrderbookProcessor, OrderbookState
from intraday.metrics import MetricsCalculator, MetricsSnapshot

print("모듈 로드 완료!")

모듈 로드 완료!


---

## 2. Orderbook (호가창) 이해하기

### 개념 설명

**Orderbook**은 특정 자산에 대한 매수(Bid)와 매도(Ask) 주문을 가격별로 정렬한 목록입니다.

```
           ORDERBOOK (호가창)              
    매수 (Bid)    |      매도 (Ask)         
   "사고 싶다"     |     "팔고 싶다"          
--------------------------------------------
 $99,950 (2.5)  |  $100,050 (1.2)  <- Best Ask
 $99,900 (1.8)  |  $100,100 (3.4)        
 $99,850 (4.2)  |  $100,150 (2.1)        
      ^         |                       
 Best Bid       |                        
```

**핵심 포인트:**
- **Best Bid**: 가장 높은 매수 가격 (시장가 매도 시 체결 가격)
- **Best Ask**: 가장 낮은 매도 가격 (시장가 매수 시 체결 가격)
- **Spread**: Best Ask - Best Bid (거래 비용)

### 실습: Binance에서 실시간 데이터 수신

Binance WebSocket을 통해 BTC/USDT Orderbook 데이터를 수신해봅시다.

In [3]:
# 데이터 수집기 초기화
processor = OrderbookProcessor(max_history=500)
metrics_calc = MetricsCalculator(max_history=500)
collected_data = []

async def collect_orderbook_data(num_samples: int = 100):
    """
    Binance에서 Orderbook 데이터를 수집합니다.
    
    Args:
        num_samples: 수집할 샘플 수 (100ms 간격이므로 100개 = 약 10초)
    """
    client = BinanceWebSocketClient("btcusdt", depth_levels=20, update_speed="100ms")
    count = 0
    
    def on_data(snapshot: OrderbookSnapshot):
        nonlocal count
        count += 1
        
        # 데이터 처리
        state = processor.update(snapshot)
        metrics = metrics_calc.calculate(state)
        collected_data.append(metrics)
        
        # 진행 상황 출력
        if count % 20 == 0:
            print(f"수집 중... {count}/{num_samples} | "
                  f"Mid: ${state.mid_price:,.2f} | "
                  f"Spread: ${state.spread:.2f}")
        
        # 목표 샘플 수 도달 시 종료
        if count >= num_samples:
            asyncio.create_task(client.disconnect())
    
    print(f"BTC/USDT Orderbook 데이터 {num_samples}개 수집 시작...")
    print(f"   (약 {num_samples * 0.1:.1f}초 소요 예상)\n")
    
    await client.connect(on_data)
    
    print(f"\n데이터 수집 완료! {len(collected_data)}개 샘플")

# 데이터 수집 실행 (약 10초)
await collect_orderbook_data(100)

BTC/USDT Orderbook 데이터 100개 수집 시작...
   (약 10.0초 소요 예상)

[Client] Connecting to wss://stream.binance.com:9443/ws/btcusdt@depth20@100ms...
[Client] Connected! Receiving BTCUSDT orderbook...
수집 중... 20/100 | Mid: $90,743.96 | Spread: $0.01
수집 중... 40/100 | Mid: $90,733.73 | Spread: $0.01
수집 중... 60/100 | Mid: $90,749.54 | Spread: $0.01
수집 중... 80/100 | Mid: $90,764.92 | Spread: $0.01
수집 중... 100/100 | Mid: $90,769.82 | Spread: $0.01

데이터 수집 완료! 100개 샘플


[Client] Disconnected.


In [4]:
# 수집된 데이터 확인
df = metrics_calc.to_dataframe()
print(f"수집된 데이터: {len(df)} rows\n")
print("처음 5개 행:")
df.head()

수집된 데이터: 100 rows

처음 5개 행:


,symbol,best_bid,best_ask,mid_price,micro_price,spread,spread_bps,bid_qty,ask_qty,imbalance,mid_price_change,micro_price_change
timestamp,,,,,,,,,,,,
2025-12-11 10:57:25.829713,BTCUSDT,90745.52,90745.53,90745.525,90745.521702,0.01,0.001102,0.95837,4.67199,-0.659571,0.0,0.000000
2025-12-11 10:57:25.929489,BTCUSDT,90745.52,90745.53,90745.525,90745.522843,0.01,0.001102,1.32850,3.34421,-0.431379,0.0,0.001141
2025-12-11 10:57:26.030092,BTCUSDT,90745.52,90745.53,90745.525,90745.522450,0.01,0.001102,1.20548,3.71453,-0.509968,0.0,-0.000393
2025-12-11 10:57:26.134338,BTCUSDT,90745.52,90745.53,90745.525,90745.522178,0.01,0.001102,1.02319,3.67429,-0.564366,0.0,-0.000272
2025-12-11 10:57:26.230240,BTCUSDT,90745.52,90745.53,90745.525,90745.522178,0.01,0.001102,1.02319,3.67429,-0.564366,0.0,0.000000


In [5]:
# 기술 통계
print("기술 통계:")
df[['mid_price', 'micro_price', 'spread', 'spread_bps', 'imbalance']].describe()

기술 통계:


,mid_price,micro_price,spread,spread_bps,imbalance
count,100.000000,100.000000,1.000000e+02,1.000000e+02,100.000000
mean,90747.630800,90747.631218,1.000000e-02,1.101957e-03,0.083686
std,11.777231,11.778607,5.222250e-12,1.430010e-07,0.656894
min,90733.725000,90733.722764,1.000000e-02,1.101688e-03,-0.999327
25%,90738.275000,90738.278831,1.000000e-02,1.101908e-03,-0.403925
50%,90745.480000,90745.475070,1.000000e-02,1.101983e-03,0.168722
75%,90751.655000,90751.659899,1.000000e-02,1.102071e-03,0.667104
max,90769.815000,90769.819207,1.000000e-02,1.102126e-03,0.997985


---

## 3. Bid-Ask Spread 분석

### 개념 설명

**Spread (스프레드)** = Best Ask - Best Bid

스프레드는 다음을 의미합니다:

1. **거래 비용**: 즉시 사고팔면 스프레드만큼 손해
2. **유동성 지표**: 스프레드가 좁을수록 유동성 높음
3. **시장 효율성**: 효율적인 시장일수록 스프레드가 좁음

**Basis Points (bps)**: 상대적 스프레드
- 1 bps = 0.01% = 0.0001
- BTC/USDT: 보통 0.1~2 bps (매우 유동적)
- 알트코인: 5~50+ bps (유동성 낮음)

In [6]:
# Spread 시계열 시각화
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Spread ($)', 'Spread (bps)'),
    vertical_spacing=0.12
)

# 절대 스프레드
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['spread'],
        mode='lines',
        name='Spread ($)',
        line=dict(color='#00D4AA', width=1.5)
    ),
    row=1, col=1
)

# 상대 스프레드 (bps)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['spread_bps'],
        mode='lines',
        name='Spread (bps)',
        line=dict(color='#FF6B6B', width=1.5)
    ),
    row=2, col=1
)

fig.update_layout(
    title='Bid-Ask Spread 시계열',
    height=500,
    template='plotly_dark',
    showlegend=True
)

fig.update_yaxes(title_text='$', row=1, col=1)
fig.update_yaxes(title_text='bps', row=2, col=1)

fig.show()

In [10]:
# Spread 분포
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['spread'],
    nbinsx=100,
    name='Spread Distribution',
    marker_color='#00D4AA'
))

# 평균선 추가
mean_spread = df['spread'].mean()
fig.add_vline(
    x=mean_spread,
    line_dash="dash",
    line_color="yellow",
    annotation_text=f"평균: ${mean_spread:.2f}"
)

fig.update_layout(
    title='Spread 분포',
    xaxis_title='Spread ($)',
    yaxis_title='빈도',
    template='plotly_dark',
    height=400
)

fig.show()

---

## 4. Mid-price vs Micro-price

### 개념 설명

**Mid-price (중간가)**
```
Mid-price = (Best Bid + Best Ask) / 2
```
- 단순 산술 평균
- 주문량을 고려하지 않음

**Micro-price (마이크로 프라이스)**
```
Micro-price = (Best Bid * Ask Qty + Best Ask * Bid Qty) / (Bid Qty + Ask Qty)
```
- 주문량으로 가중한 평균
- 주문 불균형을 반영

### 예시

```
Best Bid: $100 (10 BTC 대기)
Best Ask: $101 (1 BTC 대기)

Mid-price = ($100 + $101) / 2 = $100.50

Micro-price = ($100 * 1 + $101 * 10) / (10 + 1)
            = ($100 + $1010) / 11
            = $100.91
```

**해석**: 매도 물량(1 BTC)이 매수 물량(10 BTC)보다 적으므로, 
Micro-price가 Mid-price보다 높습니다 -> **가격 상승 압력**

In [11]:
# Mid-price vs Micro-price 비교
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Mid-price vs Micro-price', 'Micro - Mid Difference'),
    vertical_spacing=0.12
)

# 가격 비교
fig.add_trace(
    go.Scatter(
        x=df.index, y=df['mid_price'],
        mode='lines', name='Mid-price',
        line=dict(color='#4ECDC4', width=1.5)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index, y=df['micro_price'],
        mode='lines', name='Micro-price',
        line=dict(color='#FF6B6B', width=1.5)
    ),
    row=1, col=1
)

# 차이 (Micro - Mid)
diff = df['micro_price'] - df['mid_price']
colors = ['#00D4AA' if d >= 0 else '#FF6B6B' for d in diff]

fig.add_trace(
    go.Bar(
        x=df.index, y=diff,
        name='Micro - Mid',
        marker_color=colors
    ),
    row=2, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="white", row=2, col=1)

fig.update_layout(
    title='Mid-price vs Micro-price 비교',
    height=600,
    template='plotly_dark',
    showlegend=True
)

fig.update_yaxes(title_text='Price ($)', row=1, col=1)
fig.update_yaxes(title_text='Difference ($)', row=2, col=1)

fig.show()

In [13]:
# 통계 요약
diff = df['micro_price'] - df['mid_price']

print("Mid-price vs Micro-price 분석\n")
print(f"평균 Mid-price:   ${df['mid_price'].mean():,.2f}")
print(f"평균 Micro-price: ${df['micro_price'].mean():,.2f}")
print(f"\n평균 차이 (Micro - Mid): ${diff.mean():.4f}")
print(f"표준편차:                 ${diff.std():.4f}")
print(f"\nMicro > Mid 비율: {(diff > 0).mean() * 100:.1f}%")
print(f"Micro < Mid 비율: {(diff < 0).mean() * 100:.1f}%")

if diff.mean() > 0:
    print("\n해석: Micro-price가 평균적으로 높음 -> 매도 물량 < 매수 물량 -> 상승 압력")
else:
    print("\n해석: Micro-price가 평균적으로 낮음 -> 매도 물량 > 매수 물량 -> 하락 압력")

Mid-price vs Micro-price 분석

평균 Mid-price:   $90,747.63
평균 Micro-price: $90,747.63

평균 차이 (Micro - Mid): $0.0004
표준편차:                 $0.0033

Micro > Mid 비율: 52.0%
Micro < Mid 비율: 48.0%

해석: Micro-price가 평균적으로 높음 -> 매도 물량 < 매수 물량 -> 상승 압력


---

## 5. Orderbook 히트맵 시각화

### 개념 설명

**Orderbook 히트맵**은 가격별 주문량을 색상 강도로 표현합니다.

- **진한 색상**: 큰 주문량 ("벽"이라고 부름)
- **연한 색상**: 작은 주문량

트레이더들은 히트맵을 통해:
1. 지지/저항 수준 파악
2. 큰 주문(고래)의 위치 확인
3. 주문 취소/추가 패턴 관찰

In [14]:
# 현재 Orderbook 스냅샷 시각화
current_state = processor.current

if current_state:
    # 데이터 준비
    bid_prices = current_state.bid_prices[:15]
    bid_qtys = current_state.bid_quantities[:15]
    ask_prices = current_state.ask_prices[:15]
    ask_qtys = current_state.ask_quantities[:15]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('매수 (Bids)', '매도 (Asks)'),
        horizontal_spacing=0.1
    )
    
    # 매수 호가 (가로 막대)
    fig.add_trace(
        go.Bar(
            y=[f"${p:,.0f}" for p in bid_prices],
            x=bid_qtys,
            orientation='h',
            name='Bids',
            marker_color='#00D4AA',
            text=[f"{q:.3f}" for q in bid_qtys],
            textposition='outside'
        ),
        row=1, col=1
    )
    
    # 매도 호가 (가로 막대)
    fig.add_trace(
        go.Bar(
            y=[f"${p:,.0f}" for p in ask_prices],
            x=ask_qtys,
            orientation='h',
            name='Asks',
            marker_color='#FF6B6B',
            text=[f"{q:.3f}" for q in ask_qtys],
            textposition='outside'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title=f'BTC/USDT Orderbook (상위 15개 호가)<br>'
              f'<sup>Best Bid: ${current_state.best_bid[0]:,.2f} | '
              f'Best Ask: ${current_state.best_ask[0]:,.2f} | '
              f'Spread: ${current_state.spread:.2f}</sup>',
        height=600,
        template='plotly_dark',
        showlegend=False
    )
    
    fig.update_xaxes(title_text='Quantity (BTC)', row=1, col=1)
    fig.update_xaxes(title_text='Quantity (BTC)', row=1, col=2)
    
    fig.show()
else:
    print("데이터가 없습니다. 먼저 데이터를 수집하세요.")

In [15]:
# Depth Chart (누적 주문량)
if current_state:
    depth_data = processor.get_depth_chart_data(num_levels=20)
    
    fig = go.Figure()
    
    # 매수 누적 (왼쪽에서 오른쪽으로)
    fig.add_trace(go.Scatter(
        x=depth_data['bid_prices'][::-1],
        y=depth_data['bid_cumulative'][::-1],
        fill='tozeroy',
        name='Bids (누적 매수)',
        line=dict(color='#00D4AA'),
        fillcolor='rgba(0, 212, 170, 0.3)'
    ))
    
    # 매도 누적 (왼쪽에서 오른쪽으로)
    fig.add_trace(go.Scatter(
        x=depth_data['ask_prices'],
        y=depth_data['ask_cumulative'],
        fill='tozeroy',
        name='Asks (누적 매도)',
        line=dict(color='#FF6B6B'),
        fillcolor='rgba(255, 107, 107, 0.3)'
    ))
    
    # Mid-price 표시
    fig.add_vline(
        x=current_state.mid_price,
        line_dash="dash",
        line_color="yellow",
        annotation_text=f"Mid: ${current_state.mid_price:,.2f}"
    )
    
    fig.update_layout(
        title='Depth Chart (누적 주문량)',
        xaxis_title='Price ($)',
        yaxis_title='Cumulative Quantity (BTC)',
        template='plotly_dark',
        height=500
    )
    
    fig.show()

---

## 6. Order Imbalance (주문 불균형)

### 개념 설명

**Order Imbalance** = (Bid Qty - Ask Qty) / (Bid Qty + Ask Qty)

- 범위: -1 ~ +1
- **+1에 가까움**: 매수 물량 >> 매도 물량 (상승 압력)
- **-1에 가까움**: 매도 물량 >> 매수 물량 (하락 압력)
- **0 근처**: 균형 상태

In [16]:
# Imbalance 시계열
fig = go.Figure()

# 색상: 양수는 초록, 음수는 빨강
colors = ['#00D4AA' if i >= 0 else '#FF6B6B' for i in df['imbalance']]

fig.add_trace(go.Bar(
    x=df.index,
    y=df['imbalance'],
    marker_color=colors,
    name='Order Imbalance'
))

# 기준선
fig.add_hline(y=0, line_dash="solid", line_color="white", line_width=1)
fig.add_hline(y=0.3, line_dash="dash", line_color="#00D4AA", 
              annotation_text="Strong Buy (+0.3)")
fig.add_hline(y=-0.3, line_dash="dash", line_color="#FF6B6B",
              annotation_text="Strong Sell (-0.3)")

fig.update_layout(
    title='Order Imbalance 시계열',
    xaxis_title='Time',
    yaxis_title='Imbalance',
    yaxis=dict(range=[-1, 1]),
    template='plotly_dark',
    height=400
)

fig.show()

In [17]:
# Imbalance vs 가격 변화 상관관계
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['imbalance'],
    y=df['mid_price_change'],
    mode='markers',
    marker=dict(
        color=df['imbalance'],
        colorscale='RdYlGn',
        size=8,
        opacity=0.6
    ),
    name='Imbalance vs Price Change'
))

# 추세선 (선형 회귀)
z = np.polyfit(df['imbalance'].dropna(), df['mid_price_change'].dropna(), 1)
p = np.poly1d(z)
x_line = np.linspace(df['imbalance'].min(), df['imbalance'].max(), 100)

fig.add_trace(go.Scatter(
    x=x_line,
    y=p(x_line),
    mode='lines',
    line=dict(color='yellow', dash='dash'),
    name='Trend'
))

fig.update_layout(
    title='Order Imbalance vs Price Change',
    xaxis_title='Order Imbalance',
    yaxis_title='Mid-price Change ($)',
    template='plotly_dark',
    height=500
)

fig.show()

# 상관계수
corr = df['imbalance'].corr(df['mid_price_change'])
print(f"\n상관계수: {corr:.4f}")
if corr > 0.1:
    print("양의 상관관계: Imbalance가 높을수록 가격이 오르는 경향")
elif corr < -0.1:
    print("음의 상관관계: Imbalance가 높을수록 가격이 내리는 경향")
else:
    print("약한 상관관계: 단기적으로 명확한 패턴 없음")


상관계수: 0.1405
양의 상관관계: Imbalance가 높을수록 가격이 오르는 경향


---

## 요약 및 핵심 포인트

### 학습한 내용

1. **Orderbook (호가창)**
   - 매수(Bid)와 매도(Ask) 주문이 가격별로 정렬된 데이터
   - Best Bid = 최고 매수가, Best Ask = 최저 매도가

2. **Bid-Ask Spread**
   - Spread = Best Ask - Best Bid
   - 유동성 지표 (좁을수록 유동성 높음)
   - BTC/USDT는 매우 좁은 스프레드 (높은 유동성)

3. **Mid-price vs Micro-price**
   - Mid-price: 단순 중간가
   - Micro-price: 주문량 가중 중간가
   - 차이 > 0: 상승 압력, 차이 < 0: 하락 압력

4. **Order Imbalance**
   - (Bid Qty - Ask Qty) / Total
   - 주문 불균형을 수치화

### 다음 단계

실시간 대시보드에서 이 지표들을 라이브로 모니터링해보세요!

```bash
uv run python -m intraday.dashboard
```